<a href="https://colab.research.google.com/github/weixiadeng/pfam_msa/blob/main/Copy_of_MSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.4 MB/s eta 0:00:00


In [2]:
pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
from Bio import SeqIO
from Bio.Seq import Seq
import scipy
import numpy as np
import requests
from io import StringIO
from itertools import permutations
import pandas as pd
import json
import copy

In [55]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Read the fasta file from GitHub url
fa_url = "https://raw.githubusercontent.com/weixiadeng/pfam_msa/main/PF00712.fa"
fa_txt = StringIO(requests.get(fa_url).text)
# Convert the MSA to array, convert the gap to "-", all letter to uppercase
sequence_data_list = list(SeqIO.parse(fa_txt, "fasta"))
sequence_array = np.char.replace(np.char.upper(np.array(sequence_data_list)), ".", "-")

In [54]:
# Compute H(a) for each column of MSA
sequence_array_size = sequence_array.shape
column_index_a = range(sequence_array_size[1])
entropy_a = dict()
for column_a in column_index_a:
    sequence_array_count = np.unique(sequence_array[:,column_a], return_counts = True)
    sequence_array_freq = sequence_array_count[1] / sequence_array_size[0]
    column_entropy = scipy.stats.entropy(sequence_array_freq)
    entropy_a[column_a] = column_entropy

In [52]:
# Compute H(a, b) for each of two-column pair of MSA
column_index_a_b = list(permutations(range(sequence_array_size[1]), 2))
entropy_a_b = dict()
for column_a_b in column_index_a_b:
    sequence_array_count = np.unique(sequence_array[:,column_a_b],
                                     axis = 0, return_counts = True)
    sequence_array_freq = sequence_array_count[1] / sequence_array_size[0]
    column_entropy = scipy.stats.entropy(sequence_array_freq)
    entropy_a_b[column_a_b] = column_entropy

In [119]:
# Compute MI(a,b) = H(a) + H(b) - H(a, b)
MI_a_b = dict()
for column_a_b in column_index_a_b:
    MI_a_b[column_a_b] = entropy_a[column_a_b[0]] + entropy_a[column_a_b[1]] - entropy_a_b[column_a_b]

In [90]:
x_a = json.dumps(entropy_a, indent = 4)
y_a = x_a.encode('utf-8')
with open("/content/gdrive/My Drive/pfam_data/entropy_a.json", "w") as f:
    f.write(str(y_a))

In [91]:
dicts = copy.copy(entropy_a_b)
new_dct = {}
for k,v in dicts.items():
    new_dct[','.join(map(str, k))] = v

x_ab = json.dumps(new_dct, indent = 4)
y_ab = x_ab.encode('utf-8')
with open("/content/gdrive/My Drive/pfam_data/entropy_a_b.json", "w") as f:
    f.write(str(y_ab))

In [120]:
dicts = copy.copy(MI_a_b)
new_dct = {}
for k,v in dicts.items():
    new_dct[','.join(map(str, k))] = v

x_ab = json.dumps(new_dct, indent = 4)
y_ab = x_ab.encode('utf-8')
with open("/content/gdrive/My Drive/pfam_data/MI_a_b.json", "w") as f:
    f.write(str(y_ab))

In [140]:
MI_a_b_df = pd.DataFrame.from_dict(MI_a_b, orient = "index")
MI_a_b_df.reset_index(inplace = True)
with open("/content/gdrive/My Drive/pfam_data/MI_a_b.csv", "w") as f:
    MI_a_b_df.to_csv(f)